In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sqlite3

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

dirname = os.path.dirname('__file__')
db_path = os.path.join(dirname, 'data', 'MtgI.db')
connector = sqlite3.connect(db_path)
land_names = pd.read_sql(r'''SELECT DISTINCT name
FROM CardsData
WHERE type_line LIKE '%land%' AND
pioneer='legal'
ORDER BY produced_mana;''',connector)
nonland_names = pd.read_sql(r'''SELECT DISTINCT name
FROM CardsData
WHERE type_line NOT LIKE '%land%'
AND pioneer='legal'
ORDER BY color_identity,cmc;''',connector)
decklists = pd.read_sql('SELECT * FROM Decklists;',connector)

# nonland_list = []
# land_list = []

# deck_ids = decklists['deck_id'].unique()


# for id in deck_ids:
#     id_nonland_df = decklists.loc[(decklists['deck_id']==id)&(decklists['card_name'].isin(nonland_names['name']))][['main_count','card_name']]
#     id_land_df = decklists.loc[(decklists['deck_id']==id)&(decklists['card_name'].isin(nonland_names['name']))][['main_count','card_name']]
#     nonland_list.append([id_nonland_df])
#     land_list.append([id_land_df])


In [25]:
land_names=list(land_names)
nonland_names=list(nonland_names)
card_names=land_names+nonland_names

deck_archetypes = pd.read_sql(r'''SELECT deck_id, archetype
                                FROM Decks
                                INNER JOIN Tournaments ON Tournaments.tournament_id=Decks.tournament_id
                                WHERE format='PI';
    ''',connector)
deck_archetypes=deck_archetypes
print(len(deck_archetypes['archetype'].unique()))

155


In [44]:
pivot_decklists = decklists.pivot_table(values='main_count',columns='card_name',index='deck_id',fill_value=0)

data_matrix = pivot_decklists.merge(deck_archetypes, on='deck_id', how='left')
data_matrix_complete = pd.DataFrame(columns=card_names+['archetype'])
print(data_matrix_complete.columns)

data_matrix_complete = data_matrix_complete.append(data_matrix,ignore_index=True)
data_matrix_complete=data_matrix_complete.fillna(0)



Index(['Obscura Storefront', 'Evolving Wilds', 'Riveteers Overlook',
       'Fabled Passage', 'Maestros Theater', 'Brokers Hideout',
       'Cabaretti Courtyard', 'Swamp', 'Urborg, Tomb of Yawgmoth',
       'Desert of the Glorified',
       ...
       'Profound Journey', 'Vanquish the Horde', 'Zetalpa, Primal Dawn',
       'Parhelion II', 'Wakening Sun's Avatar', 'Overwhelming Splendor',
       'Flight of Equenauts', 'Angel of Deliverance',
       'Brisela, Voice of Nightmares', 'archetype'],
      dtype='object', length=8838)


C:\Users\spark\AppData\Local\Temp\ipykernel_19016\1072072210.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_matrix_complete = data_matrix_complete.append(data_matrix,ignore_index=True)


In [46]:
print(data_matrix_complete['archetype'])

0       0
1       0
2       0
3       0
4       0
       ..
6763    0
6764    0
6765    0
6766    0
6767    0
Name: archetype, Length: 6768, dtype: object
